In [150]:
import pandas as pd
import MeCab
import gensim
import numpy as np
import matplotlib.pyplot as plt
import ipadic
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [151]:
nega = pd.read_csv("./out/nega.csv")
print(nega.head())

                                             content  negaposi
0  今日、コロナ禍でいつも数人しかいない近所の電停に、修学旅行生がたーくさんいるのを見た。\n少... -0.166667
1  「わかりやすくいいきる」言動には眉に唾をつけて疑いの眼差しで対応する、というのは最低限度のコ... -0.333333
2  やりたいところではありますがちんちんぶらぶら以下省略😑\n\nコロナ禍故代表者一名のみ呼ばれ... -0.142857
3  ＜速報＞新型コロナ新規感染、熊本県内３８６人 ｜ 2022/6/2 16:07 - 熊本日日... -0.066667
4  岩手で１９１人コロナ感染 – デーリー東北デジタル - デーリー東北｜岩手県関連 https... -0.142857


In [152]:
def parse(tweet_temp):
    t = MeCab.Tagger(ipadic.MECAB_ARGS)
    temp1 = t.parse(tweet_temp)
    temp2 = temp1.split("\n")
    t_list = []
    for keitaiso in temp2:
        if keitaiso not in ["EOS", ""]:
            word, hinshi = keitaiso.split("\t")
            t_temp = [word] + hinshi.split(",")
            if len(t_temp) != 10:
                t_temp += ["*"]*(10-len(t_temp))
            t_list.append(t_temp)
    return t_list

def parse_to_df(tweet_temp):
    return pd.DataFrame(parse(tweet_temp),
                        columns=["単語","品詞","品詞細分類1",
                                 "品詞細分類2","品詞細分類3",
                                 "活用型","活用形","原形","読み","発音"])
    
def make_lda_docs(texts):
    docs = []
    for text in texts:
        df = parse_to_df(text)
        extract_df = df[(df["品詞"]+"/"+df["品詞細分類1"]).isin(["名詞/一般","名詞/固有名詞"])]
        extract_df = extract_df[extract_df["原形"]!="*"]
        doc = []
        for genkei in extract_df["原形"]:
            doc.append(genkei)
        docs.append(doc)
    return docs

In [153]:
texts = nega["content"].values
docs = make_lda_docs(texts)
print(docs[0])
dictionary = gensim.corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

['コロナ', '近所', '修学旅行', '日常']


In [154]:
print("テキスト：")
print(texts[0][:200])
print(" ")
print("LDAで読む単語：")
print(",".join(docs[0][:50]))

テキスト：
今日、コロナ禍でいつも数人しかいない近所の電停に、修学旅行生がたーくさんいるのを見た。
少しずつ日常が戻ってくるのかな。
 
LDAで読む単語：
コロナ,近所,修学旅行,日常


In [155]:
n_cluster = 4
lda = gensim.models.LdaModel(
                corpus=corpus,
                id2word=dictionary,
                num_topics=n_cluster,
                minimum_probability=0.001,
                passes=20,
                update_every=0,
                chunksize=10000,
                random_state=1
)

In [156]:
corpus_lda = lda[corpus]
arr = gensim.matutils.corpus2dense(
        corpus_lda,
        num_terms=n_cluster
).T

トピック-単語分布の可視化

In [157]:
lists = []
for i in range(n_cluster):
    temp_df = pd.DataFrame(lda.show_topic(i), columns=["word", "score"])
    temp_df["topic"] = i
    lists.append(temp_df)
topic_word_df = pd.concat(lists, ignore_index=True)

topic_word_df.to_csv("./out/topic_word_df.csv", index=False)

In [158]:
#topic_word_df["rank"] = topic_word_df.groupby("topic")["score"].rank()
#topic_word_df.pivot(index='topic', columns='rank', values='word')

In [159]:
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, "./out/pyldavis_output.html")